<h1>Predicting Star Ratings from Amazon Review Titles</h1>

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline

C:\Users\Nicholas\AppData\Local\Programs\Python\Python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
dataset=pd.read_csv('amazon_reviews_utf.csv')

In [19]:
dataset

Name  Star Rating  \
0                             So far, so good (and dry)            5   
1                     Not really a lifetime warranty...            2   
2     "THIS IS THE ONLY UMBRELLA YOU SHOULD EVER PUR...            5   
3     Most good quality umbrellas can't stand up to ...            3   
4     Decent umbrella compromised by poor-quality ri...            3   
5              it is still like new. Great construction            4   
6                                     For a tiny person            2   
7                                 Decent, a bit pricey?            4   
8           Not consistent with reviews or description.            2   
9                   Seems well-built and well-designed.            5   
10    Good Quality but Nearly Useless - Umbrella for...            2   
11    ... have to say this one is by far my favorite...            5   
12                                 classy cute umbrella            5   
13                  Well made product, excellent vendor            5   
14                                This one goes to 9...            5   
15                                              support            4   
16       Excellent umbrella- would definitely recommend            5   
17                   Umbrella has lots of good features            4   
18    NOT "super light" or lightweight. Misleading p...            2   
19                                       Nope nope nope            1   
20                             Not what I had hoped....            2   
21                                 Nice while it lasted            3   
22                           Stands up to tough weather            5   
23                                          Great size.            4   
24                          Can withstand powerful wind            5   
25                   Great product and customer service            5   
26                 Has potential - top piece unreliable            4   
27                                   well-made umbrella            5   
28                                          Good value!            5   
29                          Sturdy and Well-Constructed            4   
...                                                 ...          ...   
2370                                         Five Stars            5   
2371                                    I recommend it.            5   
2372                                         Five Stars            5   
2373                                         Five Stars            5   
2374                                           One Star            1   
2375                                         Five Stars            5   
2376  This is a very good umbrella and functions wel...            4   
2377                                         Five Stars            5   
2378                                         Five Stars            5   
2379                                         Five Stars            5   
2380                                         Five Stars            5   
2381  Great product from a company that stands behin...            5   
2382                                            5 stars            5   
2383                                         Five Stars            5   
2384                                      honest review            4   
2385                                         Five Stars            5   
2386                                         Five Stars            5   
2387                                         Five Stars            5   
2388                                         Five Stars            5   
2389                                         Five Stars            5   
2390                                         Five Stars            5   
2391                                         Five Stars            5   
2392                                            awesome            5   
2393                                         Four Stars            4   
2394               

In [101]:
len(dataset)

2400

<h2>HashingVectorizer first using 'Name' Column</h2>

In [29]:
# list of text documents
text = dataset['Name']
#input_text=[]
#input_text.append(text)
# create the transform
vectorizer = HashingVectorizer(n_features=20)
# encode document
vector = vectorizer.transform(text)
#vector = vectorizer.transform(input_text)
# summarize encoded vector
X=vector.todense()

In [17]:
print(vector.shape)
print(vector.toarray())

(2400, 20)
[[ 0.          0.          0.         ...,  0.          0.          0.        ]
 [ 0.          0.          0.         ...,  0.          0.          0.        ]
 [ 0.          0.          0.25819889 ...,  0.51639778  0.25819889  0.        ]
 ..., 
 [ 0.          0.          0.         ...,  0.          0.          0.        ]
 [ 0.          0.          0.         ..., -0.70710678  0.          0.        ]
 [ 0.          0.          0.         ...,  0.          0.          0.        ]]


In [25]:
type(X)

numpy.matrixlib.defmatrix.matrix

<h2>One hot encoding y values</h2>

In [59]:
y=dataset['Star Rating']
encoder=LabelEncoder()
encoder.fit(y)
y=encoder.transform(y)
y=np_utils.to_categorical(y)

array([[ 0.,  0.,  0.,  0.,  1.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.],
       ..., 
       [ 0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  1.,  0.]], dtype=float32)

<h1>Trying an ANN first</h1>

In [61]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [93]:
#function
def baseline_model():
    classifier=Sequential()
    # Adding the input layer and the first hidden layer
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 20))

    # Adding the second hidden layer
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))

    # Adding the output layer (Softmax because we want probabilities within 0 and 1 )
    classifier.add(Dense(units = 5, kernel_initializer = 'uniform', activation = 'softmax'))

    # Compiling the ANN (gradient descent with logarithmic loss)
    classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
    return classifier

In [ ]:
classifier = KerasClassifier(build_fn=baseline_model, epochs=100, batch_size=10, verbose=0)

In [98]:
#without wrapper
classifier=Sequential()
# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 20))

# Adding the second hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))

# Adding the output layer (Softmax because we want probabilities within 0 and 1 )
classifier.add(Dense(units = 5, kernel_initializer = 'uniform', activation = 'softmax'))

# Compiling the ANN (gradient descent with logarithmic loss)
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [99]:
# Fitting the ANN to the Training set
classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

Epoch 1/100
1920/1920 [==============================] - 1s 359us/step - loss: 1.3822 - acc: 0.8052
Epoch 2/100
1920/1920 [==============================] - 0s 117us/step - loss: 0.7201 - acc: 0.8094
Epoch 3/100
1920/1920 [==============================] - 0s 115us/step - loss: 0.6576 - acc: 0.8094
Epoch 4/100
1920/1920 [==============================] - 0s 115us/step - loss: 0.6493 - acc: 0.8094
Epoch 5/100
1920/1920 [==============================] - 0s 116us/step - loss: 0.6452 - acc: 0.8094
Epoch 6/100
1920/1920 [==============================] - 0s 132us/step - loss: 0.6426 - acc: 0.8094
Epoch 7/100
1920/1920 [==============================] - 0s 143us/step - loss: 0.6406 - acc: 0.8094
Epoch 8/100
1920/1920 [==============================] - 0s 116us/step - loss: 0.6378 - acc: 0.8094
Epoch 9/100
1920/1920 [==============================] - 0s 114us/step - loss: 0.6366 - acc: 0.8094
Epoch 10/100
1920/1920 [==============================] - 0s 116us/step - loss: 0.6351 - acc: 0.8094

In [184]:
y_pred = classifier.predict(X_test)

In [100]:
score = classifier.evaluate(X_test, y_test)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

480/480 [==============================] - 0s 356us/step
Test loss: 0.61411759456
Test accuracy: 0.820833333333


<h3>Testing on made-up data</h3>

In [30]:
fake=['This umbrella sucks', 'I really like this! Great', 'Amazing, except for the handle', 'I wish this were larger', 'Works great on rainy days but it\'s too bulky', 'Never fails. EVER', 'adfhasdfjlasdklfjalsd;fkalskfjas;dkfjasl;']
fake_vector=vectorizer.transform(fake)
fake_matrix=fake_vector.todense()

In [159]:
fake_predictions=classifier.predict(fake_matrix)
fake_predictions

array([[ 0.01485564,  0.01612477,  0.01245042,  0.08732177,  0.86924744],
       [ 0.04533625,  0.03064492,  0.02977685,  0.14688113,  0.74736083],
       [ 0.06426573,  0.03704768,  0.03891603,  0.17027591,  0.68949461],
       [ 0.07903251,  0.04129742,  0.04553317,  0.18487364,  0.64926326],
       [ 0.01769844,  0.01791328,  0.01432108,  0.09516492,  0.85490227],
       [ 0.21544071,  0.06371476,  0.09276783,  0.24571601,  0.38236073],
       [ 0.0771531 ,  0.04079749,  0.04472519,  0.18318509,  0.6541391 ]], dtype=float32)

<strong>Unfortunately it seems that our classifier is biased too much on the positive reviews. Now we try oversampling using SMOTE.</strong>

In [5]:
from imblearn.over_sampling import SMOTE 

sm=SMOTE(random_state=1, ratio='auto', k_neighbors=5, m_neighbors=10, 
         out_step=0.5, kind='regular', n_jobs=-1)
y=dataset['Star Rating']
X_res, y_res= sm.fit_sample(X, y)
encoder=LabelEncoder()
encoder.fit(y_res)
y_res=encoder.transform(y_res)
y_res=np_utils.to_categorical(y_res)

NameError: name 'dataset' is not defined

In [182]:
new_classifier= KerasClassifier(build_fn=baseline_model, epochs=100, batch_size=10, verbose=0)
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size = 0.2, random_state = 0)
new_classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

C:\Users\Nicholas\AppData\Local\Programs\Python\Python36\lib\site-packages\tensorflow\python\util\tf_inspect.py:55: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))


In [29]:
fake=['This umbrella sucks', 'I really like this! Great', 'Amazing, except for the handle', 'I wish this were larger', 'Works great on rainy days but it\'s too bulky', 'Never fails. EVER', 'adfhasdfjlasdklfjalsd;fkalskfjas;dkfjasl;']

In [183]:
fake_predictions_2=new_classifier.predict(fake_matrix)
fake_predictions_2

array([2, 1, 2, 1, 1, 0, 0])

In [175]:
kfold = KFold(n_splits=3, shuffle=True, random_state=0)
results = cross_val_score(new_classifier, X_res, y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

C:\Users\Nicholas\AppData\Local\Programs\Python\Python36\lib\site-packages\tensorflow\python\util\tf_inspect.py:55: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))


Baseline: 67.22% (4.01%)


<h2>Running same model more times on same data</h2>

In [70]:
classifier.fit(X_train, y_train, batch_size = 10, epochs = 200)

Epoch 1/200
1920/1920 [==============================] - 0s 118us/step - loss: 0.5431 - acc: 0.8229
Epoch 2/200
1920/1920 [==============================] - 0s 101us/step - loss: 0.5424 - acc: 0.8224
Epoch 3/200
1920/1920 [==============================] - 0s 99us/step - loss: 0.5424 - acc: 0.8234
Epoch 4/200
1920/1920 [==============================] - 0s 99us/step - loss: 0.5420 - acc: 0.8229
Epoch 5/200
1920/1920 [==============================] - 0s 103us/step - loss: 0.5419 - acc: 0.8245
Epoch 6/200
1920/1920 [==============================] - 0s 127us/step - loss: 0.5413 - acc: 0.8240
Epoch 7/200
1920/1920 [==============================] - 0s 104us/step - loss: 0.5414 - acc: 0.8224
Epoch 8/200
1920/1920 [==============================] - 0s 102us/step - loss: 0.5408 - acc: 0.8245
Epoch 9/200
1920/1920 [==============================] - 0s 104us/step - loss: 0.5416 - acc: 0.8240
Epoch 10/200
1920/1920 [==============================] - 0s 103us/step - loss: 0.5412 - acc: 0.8250
E

1920/1920 [==============================] - 0s 104us/step - loss: 0.5194 - acc: 0.8193
Epoch 162/200
1920/1920 [==============================] - 0s 102us/step - loss: 0.5191 - acc: 0.8187
Epoch 163/200
1920/1920 [==============================] - 0s 102us/step - loss: 0.5195 - acc: 0.8214
Epoch 164/200
1920/1920 [==============================] - 0s 102us/step - loss: 0.5200 - acc: 0.8214
Epoch 165/200
1920/1920 [==============================] - 0s 102us/step - loss: 0.5187 - acc: 0.8177
Epoch 166/200
1920/1920 [==============================] - 0s 102us/step - loss: 0.5188 - acc: 0.8203
Epoch 167/200
1920/1920 [==============================] - 0s 101us/step - loss: 0.5188 - acc: 0.8182
Epoch 168/200
1920/1920 [==============================] - 0s 102us/step - loss: 0.5184 - acc: 0.8208
Epoch 169/200
1920/1920 [==============================] - 0s 101us/step - loss: 0.5192 - acc: 0.8198
Epoch 170/200
1920/1920 [==============================] - 0s 107us/step - loss: 0.5188 - acc: 0

<strong>This results in a worse accuracy (81%)
    
To get better results we need more data and possibly a component from emotional analysis. </strong>

<h1>K-Fold</h1>

In [95]:
kfold = KFold(n_splits=10, shuffle=True, random_state=0)
results = cross_val_score(classifier, X, y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Baseline: 83.67% (2.38%)


<h2>Saving model</h2>

In [102]:
model_json = classifier.to_json()
with open("AmazonRatingANN.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
classifier.save_weights("AmazonRatingANN.h5")
print("Saved model to disk")

Saved model to disk


<h1>Unanswered questions</h1>
1. How to choose the right hash size
2. How to actually use this in a business problem
3. How to get emotions for each input

<h1>SOM</h1>

In [103]:
from mvpa2.suite import *

Failed to import duecredit due to No module named 'duecredit'


 * Please note: warnings are printed only once, but underlying problem might occur many times *


C:\Users\Nicholas\AppData\Local\Programs\Python\Python36\lib\site-packages\scipy\integrate\quadpack.py:364: IntegrationWarning: Extremely bad integrand behavior occurs at some points of the
  integration interval.
  warnings.warn(msg, IntegrationWarning)


RuntimeError: module compiled against API version 0xc but this version of numpy is 0xb

C:\Users\Nicholas\AppData\Local\Programs\Python\Python36\lib\site-packages\mvpa2\datasets\sources\skl_data.py:32: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  argnames, varargs, varkw, defaults = inspect.getargspec(fx)
<string>:60: DeprecationWarning: invalid escape sequence \#
<string>:59: DeprecationWarning: invalid escape sequence \#
<string>:59: DeprecationWarning: invalid escape sequence \#
<string>:65: DeprecationWarning: invalid escape sequence \c


In [107]:
X_arr=vector.toarray()

In [108]:
som = SimpleSOMMapper((20, 30), 400, learning_rate=0.05)
som.train(X_arr)

In [111]:
som.K

array([[[ -1.63231430e-01,  -2.12023976e-02,   2.64967467e-01, ...,
           1.08223717e-01,   2.02714840e-01,   5.39986404e-02],
        [ -1.04001559e-01,   3.76143380e-02,   3.80886612e-01, ...,
           6.67869201e-02,   7.71265617e-02,   6.63713425e-02],
        [ -2.00105897e-02,   7.89084503e-03,   5.86958105e-01, ...,
          -7.00586829e-04,  -2.96551813e-02,   1.64379571e-01],
        ..., 
        [ -2.47743671e-02,  -6.10222786e-01,   9.02851233e-02, ...,
          -1.62993410e-02,   4.62192227e-01,  -4.84200686e-02],
        [ -3.73261346e-02,  -3.81609517e-01,   2.13799608e-01, ...,
          -2.56572293e-02,   2.03346929e-01,  -6.22650067e-02],
        [ -1.25713012e-01,  -1.57983132e-01,   2.03186833e-01, ...,
           8.70507170e-02,   2.02088132e-01,   9.80057493e-03]],

       [[ -1.54760709e-01,  -6.79466693e-03,   3.08744127e-01, ...,
           1.34084508e-01,   3.59844832e-01,   6.13661604e-02],
        [ -5.68673121e-02,   8.23532871e-02,   5.50610991e-0

In [190]:
def sammon(x, n = 2, display = 2, inputdist = 'raw', maxhalves = 20, maxiter = 500, tolfun = 1e-9, init = 'pca'):

    import numpy as np 
    from scipy.spatial.distance import cdist

    """Perform Sammon mapping on dataset x

    y = sammon(x) applies the Sammon nonlinear mapping procedure on
    multivariate data x, where each row represents a pattern and each column
    represents a feature.  On completion, y contains the corresponding
    co-ordinates of each point on the map.  By default, a two-dimensional
    map is created.  Note if x contains any duplicated rows, SAMMON will
    fail (ungracefully). 

    [y,E] = sammon(x) also returns the value of the cost function in E (i.e.
    the stress of the mapping).

    An N-dimensional output map is generated by y = sammon(x,n) .

    A set of optimisation options can be specified using optional
    arguments, y = sammon(x,n,[OPTS]):

       maxiter        - maximum number of iterations
       tolfun         - relative tolerance on objective function
       maxhalves      - maximum number of step halvings
       input          - {'raw','distance'} if set to 'distance', X is 
                        interpreted as a matrix of pairwise distances.
       display        - 0 to 2. 0 least verbose, 2 max verbose.
       init           - {'pca', 'random'}

    The default options are retrieved by calling sammon(x) with no
    parameters.

    File        : sammon.py
    Date        : 18 April 2014
    Authors     : Tom J. Pollard (tom.pollard.11@ucl.ac.uk)
                : Ported from MATLAB implementation by 
                  Gavin C. Cawley and Nicola L. C. Talbot

    Description : Simple python implementation of Sammon's non-linear
                  mapping algorithm [1].

    References  : [1] Sammon, John W. Jr., "A Nonlinear Mapping for Data
                  Structure Analysis", IEEE Transactions on Computers,
                  vol. C-18, no. 5, pp 401-409, May 1969.

    Copyright   : (c) Dr Gavin C. Cawley, November 2007.

    This program is free software; you can redistribute it and/or modify
    it under the terms of the GNU General Public License as published by
    the Free Software Foundation; either version 2 of the License, or
    (at your option) any later version.

    This program is distributed in the hope that it will be useful,
    but WITHOUT ANY WARRANTY; without even the implied warranty of
    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
    GNU General Public License for more details.

    You should have received a copy of the GNU General Public License
    along with this program; if not, write to the Free Software
    Foundation, Inc., 59 Temple Place, Suite 330, Boston, MA 02111-1307 USA

    """
#
#    def euclid(a,b):
#        d = np.sqrt( ((a**2).sum(axis=1)*np.ones([1,b.shape[0]]).T).T + \
#            np.ones([a.shape[0],1])*(b**2).sum(axis=1)-2*(np.dot(a,b.T)))
#        return d

    X = x

    # Create distance matrix unless given by parameters
    if inputdist == 'distance':
        xD = X
    else:
        xD = cdist(X, X)

    # Remaining initialisation
    N = X.shape[0] # hmmm, shape[1]?
    scale = 0.5 / xD.sum()

    if init == 'pca':
        [UU,DD,_] = np.linalg.svd(X)
        Y = UU[:,:n]*DD[:n] 
    else:
        Y = np.random.normal(0.0,1.0,[N,n])
    one = np.ones([N,n])

    xD = xD + np.eye(N)        
    xDinv = 1 / xD # Returns inf where D = 0.
    xDinv[np.isinf(xDinv)] = 0 # Fix by replacing inf with 0 (default Matlab behaviour).    
    yD = cdist(Y, Y) + np.eye(N)
    yDinv = 1. / yD # Returns inf where d = 0. 
    
    np.fill_diagonal(xD, 1)    
    np.fill_diagonal(yD, 1)
    np.fill_diagonal(xDinv, 0)
    np.fill_diagonal(yDinv, 0)
    
    xDinv[np.isnan(xDinv)] = 0
    yDinv[np.isnan(xDinv)] = 0
    xDinv[np.isinf(xDinv)] = 0    
    yDinv[np.isinf(yDinv)] = 0 # Fix by replacing inf with 0 (default Matlab behaviour).
    
    delta = xD - yD 
    E = ((delta**2)*xDinv).sum() 

    # Get on with it
    for i in range(maxiter):

        # Compute gradient, Hessian and search direction (note it is actually
        # 1/4 of the gradient and Hessian, but the step size is just the ratio
        # of the gradient and the diagonal of the Hessian so it doesn't
        # matter).
        delta = yDinv - xDinv
        deltaone = np.dot(delta,one)
        g = np.dot(delta, Y) - (Y * deltaone)
        dinv3 = yDinv ** 3
        y2 = Y ** 2
        H = np.dot(dinv3,y2) - deltaone - np.dot(2, Y) * np.dot(dinv3, Y) + y2 * np.dot(dinv3,one)
        s = -g.flatten(order='F') / np.abs(H.flatten(order='F'))
        y_old = Y

        # Use step-halving procedure to ensure progress is made
        for j in range(maxhalves):
            s_reshape = s.reshape(2,len(s)//2).T 
            #s_reshape = s.reshape(-1).T
            y = y_old + s_reshape
            d = cdist(y, y) + np.eye(N)
            dinv = 1 / d # Returns inf where D = 0. 
            dinv[np.isinf(dinv)] = 0 # Fix by replacing inf with 0 (default Matlab behaviour).
            delta = xD - d
            E_new = ((delta**2)*xDinv).sum()
            if E_new < E:
                break
            else:
                s = np.dot(0.5,s)

        # Bomb out if too many halving steps are required
        if j == maxhalves:
            print('Warning: maxhalves exceeded. Sammon mapping may not converge...')

        # Evaluate termination criterion
        if np.abs((E - E_new) / E) < tolfun:
            if display:
                print('TolFun exceeded: Optimisation terminated')
            break

        # Report progress
        E = E_new
        if display > 1:
            print('epoch = ' + str(i) + ': E = ' + str(E * scale))

    # Fiddle stress to match the original Sammon paper
    E = E * scale
    
    return [y,E]

In [124]:
som.K.shape

(20, 30, 20)

In [136]:
X.shape

(2400, 20)

In [195]:
y_sam, E=sammon(X, init='random')

C:\Users\Nicholas\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:90: RuntimeWarning: divide by zero encountered in true_divide


epoch = 0: E = 0.284084295351
epoch = 1: E = 0.282075886171
epoch = 2: E = 0.284171160353
epoch = 3: E = 0.284084295351
epoch = 4: E = 0.282075886171
epoch = 5: E = 0.284171160353
epoch = 6: E = 0.284084295351
epoch = 7: E = 0.282075886171
epoch = 8: E = 0.284171160353
epoch = 9: E = 0.284084295351
epoch = 10: E = 0.282075886171
epoch = 11: E = 0.284171160353
epoch = 12: E = 0.284084295351
epoch = 13: E = 0.282075886171
epoch = 14: E = 0.284171160353
epoch = 15: E = 0.284084295351
epoch = 16: E = 0.282075886171
epoch = 17: E = 0.284171160353
epoch = 18: E = 0.284084295351
epoch = 19: E = 0.282075886171
epoch = 20: E = 0.284171160353
epoch = 21: E = 0.284084295351
epoch = 22: E = 0.282075886171
epoch = 23: E = 0.284171160353
epoch = 24: E = 0.284084295351
epoch = 25: E = 0.282075886171
epoch = 26: E = 0.284171160353
epoch = 27: E = 0.284084295351
epoch = 28: E = 0.282075886171
epoch = 29: E = 0.284171160353
epoch = 30: E = 0.284084295351
epoch = 31: E = 0.282075886171
epoch = 32: E = 0.

epoch = 260: E = 0.284171160353
epoch = 261: E = 0.284084295351
epoch = 262: E = 0.282075886171
epoch = 263: E = 0.284171160353
epoch = 264: E = 0.284084295351
epoch = 265: E = 0.282075886171
epoch = 266: E = 0.284171160353
epoch = 267: E = 0.284084295351
epoch = 268: E = 0.282075886171
epoch = 269: E = 0.284171160353
epoch = 270: E = 0.284084295351
epoch = 271: E = 0.282075886171
epoch = 272: E = 0.284171160353
epoch = 273: E = 0.284084295351
epoch = 274: E = 0.282075886171
epoch = 275: E = 0.284171160353
epoch = 276: E = 0.284084295351
epoch = 277: E = 0.282075886171
epoch = 278: E = 0.284171160353
epoch = 279: E = 0.284084295351
epoch = 280: E = 0.282075886171
epoch = 281: E = 0.284171160353
epoch = 282: E = 0.284084295351
epoch = 283: E = 0.282075886171
epoch = 284: E = 0.284171160353
epoch = 285: E = 0.284084295351
epoch = 286: E = 0.282075886171
epoch = 287: E = 0.284171160353
epoch = 288: E = 0.284084295351
epoch = 289: E = 0.282075886171
epoch = 290: E = 0.284171160353
epoch = 

In [203]:
plt.scatter(y_sam[y==0, 0], y_sam[y ==0, 1], s=20, c='r', marker='o',label=0)
plt.scatter(y_sam[y ==1, 0], y_sam[y ==1, 1], s=20, c='b', marker='D',label=1)
plt.scatter(y_sam[y ==2, 0], y_sam[y ==2, 1], s=20, c='y', marker='v',label=2)
plt.title('Sammon wtf')
plt.legend(loc=2)
plt.show()

In [198]:
y_sam

array([[ 0.45639169,  0.07408636],
       [ 1.92768422,  0.25182436],
       [-0.71361493,  0.27284756],
       ..., 
       [-0.68951068, -0.47527581],
       [-0.23592119, -0.20685205],
       [-0.31193699,  2.23774418]])

In [208]:
sammon_df=pd.DataFrame(y_sam)
sammon_df.to_csv('sammon_df.csv', index=False)

In [205]:
E=0.28207588617081786

"Neural networks are a good example of where going from the model form to
the fitted model is difficult. We must use numerical methods, and convergence to a global optimum
is not guaranteed. This means that to find the “best” fitted model (i.e., the model with the highest
criterion value) we need multiple random starts."

<i>I'm just going to give up on sammon/ SOMs for text classification for now...</i>

In [2]:
training_raw=pd.read_json('reviews_Video_Games_5.json', lines=True)

In [8]:
training_raw.columns

Index(['asin', 'helpful', 'overall', 'reviewText', 'reviewTime', 'reviewerID',
       'reviewerName', 'summary', 'unixReviewTime'],
      dtype='object')

In [12]:
training_raw.head()

asin  helpful  overall  \
0  0700099867  [8, 12]        1   
1  0700099867   [0, 0]        4   
2  0700099867   [0, 0]        1   
3  0700099867  [7, 10]        3   
4  0700099867   [2, 2]        4   

                                          reviewText   reviewTime  \
0  Installing the game was a struggle (because of...   07 9, 2012   
1  If you like rally cars get this game you will ...  06 30, 2013   
2  1st shipment received a book instead of the ga...  06 28, 2014   
3  I got this version instead of the PS3 version,...  09 14, 2011   
4  I had Dirt 2 on Xbox 360 and it was an okay ga...  06 14, 2011   

       reviewerID                        reviewerName  \
0  A2HD75EMZR8QLN                                 123   
1  A3UR8NLLY1ZHCX  Alejandro Henao "Electronic Junky"   
2  A1INA0F5CWW3J4          Amazon Shopper "Mr.Repsol"   
3  A1DLMTOTHQ4AST                            ampgreen   
4  A361M14PU2GUEG        Angry Ryan "Ryan A. Forrest"   

                                           summary  unixReviewTime  
0         Pay to unlock content? I don't think so.      1341792000  
1                                  Good rally game      1372550400  
2                                        Wrong key      1403913600  
3  awesome game, if it did not crash frequently !!      1315958400  
4                                           DIRT 3      1308009600

In [34]:
X=training_raw['reviewText'].iloc[0:50000]

In [35]:
y=training_raw['overall'].iloc[0:50000]

In [17]:
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
from keras.layers import Dropout

In [7]:
negative = ['not', 'neither', 'nor', 'but', 'however', 'although', 'nonetheless', 'despite', 'except',
                         'even though', 'yet']
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)

In [8]:
def clean(doc):
    lemma=WordNetLemmatizer()
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop if i not in negative])
    punc_free = "".join([ch for ch in stop_free if ch not in exclude])
    normalized = " ".join([lemma.lemmatize(word) for word in punc_free.split()])
    return normalized

In [36]:
for index, item in enumerate(X):
    X[index]=clean(item)

In [37]:
# create the transform
vectorizer = HashingVectorizer()
# encode document
vector = vectorizer.transform(X)
#vector = vectorizer.transform(input_text)
# summarize encoded vector
X_mat=vector.todense()

MemoryError: 

In [32]:
n_features=2**20

In [13]:
X_res, y_res= sm.fit_sample(X, y)
encoder=LabelEncoder()
encoder.fit(y_res)
y_res=encoder.transform(y_res)
y_res=np_utils.to_categorical(y_res)

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size = 0.2, random_state = 0)

In [19]:
#without wrapper
classifier=Sequential()
# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 20))

#Adding dropout
classifier.add(Dropout(0.4, seed=0))

# Adding the second hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))

#Adding the third hidden layer
classifier.add(Dense(units=10, kernel_initializer='uniform', activation='relu'))

#Adding dropout
classifier.add(Dropout(0.4, seed=0))

# Adding the output layer (Softmax because we want probabilities within 0 and 1 )
classifier.add(Dense(units = 5, kernel_initializer = 'uniform', activation = 'softmax'))

# Compiling the ANN (gradient descent with logarithmic loss)
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [28]:
classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

Epoch 1/100
104224/104224 [==============================] - 14s 134us/step - loss: 1.6096 - acc: 0.1985
Epoch 2/100
104224/104224 [==============================] - 14s 131us/step - loss: 1.6095 - acc: 0.2017
Epoch 3/100
104224/104224 [==============================] - 13s 129us/step - loss: 1.6096 - acc: 0.2001
Epoch 4/100
104224/104224 [==============================] - 14s 130us/step - loss: 1.6096 - acc: 0.2007
Epoch 5/100
104224/104224 [==============================] - 14s 131us/step - loss: 1.6096 - acc: 0.1993
Epoch 6/100
104224/104224 [==============================] - 14s 131us/step - loss: 1.6096 - acc: 0.1995
Epoch 7/100
104224/104224 [==============================] - 14s 131us/step - loss: 1.6096 - acc: 0.1981
Epoch 8/100
104224/104224 [==============================] - 14s 132us/step - loss: 1.6096 - acc: 0.1995
Epoch 9/100
104224/104224 [==============================] - 14s 131us/step - loss: 1.6096 - acc: 0.1999
Epoch 10/100
104224/104224 [===========================

104224/104224 [==============================] - 15s 148us/step - loss: 1.6096 - acc: 0.2004
Epoch 79/100
104224/104224 [==============================] - 14s 133us/step - loss: 1.6096 - acc: 0.1996
Epoch 80/100
104224/104224 [==============================] - 14s 133us/step - loss: 1.6096 - acc: 0.1995
Epoch 81/100
104224/104224 [==============================] - 14s 132us/step - loss: 1.6096 - acc: 0.1989
Epoch 82/100
104224/104224 [==============================] - 14s 132us/step - loss: 1.6097 - acc: 0.1967
Epoch 83/100
104224/104224 [==============================] - 14s 132us/step - loss: 1.6096 - acc: 0.1992
Epoch 84/100
104224/104224 [==============================] - 14s 132us/step - loss: 1.6096 - acc: 0.2001
Epoch 85/100
104224/104224 [==============================] - 14s 132us/step - loss: 1.6096 - acc: 0.1972
Epoch 86/100
104224/104224 [==============================] - 14s 131us/step - loss: 1.6096 - acc: 0.2006
Epoch 87/100
104224/104224 [==============================]

In [31]:
fake_predictions=classifier.predict(fake_matrix)
fake_predictions

array([[0.20486134, 0.20371065, 0.19212134, 0.20246992, 0.19683681],
       [0.20486134, 0.20371065, 0.19212134, 0.20246992, 0.19683681],
       [0.20486134, 0.20371065, 0.19212134, 0.20246992, 0.19683681],
       [0.20486134, 0.20371065, 0.19212134, 0.20246992, 0.19683681],
       [0.20486134, 0.20371065, 0.19212134, 0.20246992, 0.19683681],
       [0.20486134, 0.20371065, 0.19212134, 0.20246992, 0.19683681],
       [0.20486134, 0.20371065, 0.19212134, 0.20246992, 0.19683681]],
      dtype=float32)

In [23]:
import tensorflow as tf

In [25]:
import keras

In [27]:
config = tf.ConfigProto(device_count={"CPU":  4})
keras.backend.tensorflow_backend.set_session(tf.Session(config=config))